In [1]:
# Initialization
import pyhdk 
import pandas
import pyarrow as pa

hdk = pyhdk.hdk.HDK(
    debug_logs="INFO", # generates log file, DEBUG2 for more verbosity 
) 

In [2]:
tbl = pa.Table.from_pandas(pandas.DataFrame({"a": [1, 2, 3], "b": [10, 20, 30]}))
hdk_tbl = hdk.import_arrow(tbl, "test")

In [ ]:
# Basic query
print(hdk.sql("SELECT * FROM test;").to_arrow().to_pandas())

print(hdk.sql("SELECT a, count(*), sum(b) FROM test GROUP BY a;").to_arrow().to_pandas())

# Cleanup
hdk.drop_table("test")